## 🧼 Notebook 1: Preprocesamiento de reseñas

En esta notebook se realiza la carga, limpieza y normalización inicial de los datos de reseñas recolectadas mediante scraping. Se unifican los datasets, se corrigen las fechas, se limpian los textos y se eliminan duplicados o valores nulos antes del análisis exploratorio.

## 1. Imports y configuración general

In [ ]:
# Imports y configuración general
import pandas as pd
import json
from datetime import datetime
import re
from unidecode import unidecode
import os
from dateparser import parse

## 2. Configuración del entorno y rutas

Se establecen las configuraciones iniciales del entorno:
- Se declaran las rutas a los datos crudos y procesados. Estas rutas permiten guardar versiones intermedias o finales del dataset.

In [ ]:
# Configuración de rutas
data_raw_dir = '../data/raw'
data_processed_dir = '../data/processed'

# Crear estructura de outputs
output_dirs = [
    '../outputs',
    '../outputs/visualizations', 
    '../outputs/analysis_results',
    '../outputs/exports',
    '../outputs/dashboards'
]

for dir_path in output_dirs:
    os.makedirs(dir_path, exist_ok=True)

## 3. Función de carga y limpieza de reseñas

La función `load_and_clean()` permite cargar reseñas desde un archivo `.json` y aplicar una limpieza básica.  
Incluye los siguientes pasos:

- **Carga del JSON** y conversión a DataFrame.
- **Normalización del texto** (pasaje a minúsculas, remoción de tildes y puntuación).
- **Conversión de fechas** desde formatos ISO o con nombres de meses en español, utilizando `dateparser`.
- Asignación del nombre del producto y retorno del DataFrame limpio.

Esta función permite reutilizar el mismo proceso para distintos productos.

In [ ]:
def load_and_clean(filename, product_name):
    """
    Carga y limpia reseñas desde un archivo JSON.
    
    Realiza preprocesamiento completo incluyendo limpieza de texto,
    normalización de fechas, validación de datos y creación de 
    variables auxiliares para análisis posterior.
    
    Args:
        filename (str): Nombre del archivo JSON en data/raw
        product_name (str): Nombre del producto para identificación
    
    Returns:
        pd.DataFrame: DataFrame limpio con las reseñas procesadas
                     Columnas: text, rating, date, useful_votes, producto,
                              text_clean, text_length, year_month
    
    Raises:
        FileNotFoundError: Si el archivo JSON no existe
        ValueError: Si el JSON tiene estructura incorrecta
    
    Example:
        >>> df = load_and_clean('comentarios_Samsung_A15.json', 'Samsung A15')
        >>> print(len(df))  # Número de reseñas procesadas
    """
    # Cargar datos desde JSON
    file_path = os.path.join(data_raw_dir, filename)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Convertir a DataFrame
    df = pd.DataFrame(data)
    
    # Normalizar texto (minúsculas, sin tildes, sin puntuación)
    def clean_text(text):
        if pd.isna(text):
            return ''
        text = str(text).lower()
        text = unidecode(text)  # Remover tildes
        text = re.sub(r'[^\w\s]', ' ', text)  # Remover puntuación
        text = re.sub(r'\s+', ' ', text).strip()  # Normalizar espacios
        return text
    
    # Aplicar limpieza de texto
    df['text_clean'] = df['text'].apply(clean_text)
    
    # Calcular longitud del texto
    df['text_length'] = df['text'].fillna('').str.len()
    
    # Convertir fechas usando dateparser (maneja formatos en español)
    df['date'] = df['date'].apply(lambda x: parse(str(x), languages=['es']).strftime('%Y-%m-%d') if parse(str(x), languages=['es']) else None)
    df['date'] = pd.to_datetime(df['date'])
    
    # Crear columna de año-mes para agrupaciones
    df['year_month'] = df['date'].dt.to_period('M')
    
    # Agregar identificador del producto
    df['producto'] = product_name
    
    print(f"✅ {product_name}: {len(df)} reseñas cargadas y procesadas")
    return df

## 4. Carga de datasets por producto

Se cargan las reseñas de cada producto desde los archivos JSON generados por el scraper.
Esto nos permite mantener trazabilidad del origen de cada reseña y preparar los datos para su posterior unificación.

In [ ]:
# Cargar datasets por producto
df_samsung = load_and_clean('comentarios_Samsung_A15.json', 'Samsung A15')
df_motorola = load_and_clean('comentarios_Motorola_G32.json', 'Motorola G32')

## 5. Unificación, deduplicación y control de nulos

Una vez cargadas y limpiadas las reseñas de cada producto:

- Se concatenan en un único DataFrame (`df_total`).
- Se eliminan duplicados basados en el texto de la reseña.
- Se eliminan filas con valores nulos en campos clave como `text` y `rating`.

Se imprime un resumen de los valores nulos y se muestra un preview del DataFrame final.

In [ ]:
# Unificar datasets
df_total = pd.concat([df_samsung, df_motorola], ignore_index=True)

# Eliminar duplicados
df_total.drop_duplicates(subset=['text_clean', 'producto'], inplace=True)

print(f"📊 Total de reseñas después de deduplicación: {len(df_total)}")

### 5.1 Verificación del estado del DataFrame final

Se exploran aspectos clave del dataset:

- Cantidad de fechas faltantes.
- Cantidad total de valores nulos por columna.
- Distribución de las calificaciones (`rating`), útil para entender posibles sesgos.
- Estadísticas descriptivas generales (`describe()`), que permiten anticipar outliers o errores de carga.

Este chequeo asegura que los datos están listos para pasar al análisis exploratorio (EDA).

In [ ]:
# Verificación de calidad de datos
print("→ Información general del dataset:")
print(df_total.info())

print("\n→ Valores nulos por columna:")
print(df_total.isnull().sum())

print("\n→ Distribución de ratings:")
print(df_total['rating'].value_counts().sort_index())

print("\n→ Muestra del dataset final:")
print(df_total.head())

## 6. Guardado del dataset limpio

Se exportan las reseñas unificadas y procesadas a un archivo `.csv` para análisis exploratorio posterior.  
Este archivo puede ser utilizado en notebooks siguientes para visualizaciones, NLP u otros análisis.

In [ ]:
# Crear directorio de salida si no existe
os.makedirs(data_processed_dir, exist_ok=True)

# Guardar en data/processed (datos principales)
output_path = os.path.join(data_processed_dir, 'reviews_unificado.csv')
df_total.to_csv(output_path, index=False, encoding='utf-8')

# Guardar copia en outputs/analysis_results (para fácil acceso)
output_path_analysis = '../outputs/analysis_results/00_reviews_unificado.csv'
df_total.to_csv(output_path_analysis, index=False, encoding='utf-8')

print(f"✅ Datos guardados en: {output_path}")
print(f"✅ Copia guardada en: {output_path_analysis}")
print(f"📊 Total de reseñas procesadas: {len(df_total)}")